In [15]:
import numpy as np
import cv2
import mediapipe as mp
import time
import matplotlib.pyplot as plt

In [13]:
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

In [14]:
curr_t=0
prev_t=0
tips=[4,8,12,16,20,25,29,33,37,41]
video = cv2.VideoCapture(0)
while True:
    ret, frame = video.read()
    imgRGB=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results=hands.process(imgRGB)
    #print(results.multi_hand_landmarks)
    lmList=[]
    if results.multi_handedness:
        label = results.multi_handedness[0].classification[0].label  # label gives if hand is left or right
           
    if results.multi_hand_landmarks:
        for hd in results.multi_hand_landmarks:
            for idx,lm in enumerate(hd.landmark):
                h,w,c=frame.shape
                cx,cy=int(lm.x*w),int(lm.y*h)
                lmList.append([idx,cx,cy,label]) 
    
        #mpDraw.draw_landmarks(frame,hd,mpHands.HAND_CONNECTIONS)
    total=0
    if ((len(lmList)!=0) and (len(lmList)<=21)):
        if ((lmList[tips[0]][1]<lmList[tips[0]-1][1]) and (lmList[tips[0]][3]=='Right')):
            total+=1
        elif ((lmList[tips[0]][1]>lmList[tips[0]-1][1]) and (lmList[tips[0]][3]=='Left')):
            total+=1
        for i in range(1,5):
            if ((lmList[tips[i]][2]<lmList[tips[i]-2][2])):
                total+=1
            
    elif ((len(lmList)!=0) and (len(lmList)>21) and (len(lmList)<=42)):
        if ((lmList[tips[0]][1]<lmList[tips[0]-1][1]) and (lmList[tips[0]][3]=='Right')):
            total+=1
        elif ((lmList[tips[0]][1]>lmList[tips[0]-1][1]) and (lmList[tips[0]][3]=='Left')):
            total+=1
        for i in range(1,5):
            if ((lmList[tips[i]][2]<lmList[tips[i]-2][2])):
                total+=1
        if ((lmList[tips[5]][1]>lmList[tips[5]-1][1]) and (lmList[tips[0]][3]=='Right')):
            total+=1
        elif ((lmList[tips[5]][1]<lmList[tips[5]-1][1]) and (lmList[tips[0]][3]=='Left')):
            total+=1
        for i in range(6,10):
            if ((lmList[tips[i]][2]<lmList[tips[i]-2][2])):
                total+=1
    curr_t=time.time()
    fps=1/(curr_t-prev_t)
    prev_t=curr_t
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
      
    
    cv2.putText(frame, str(total), (10,78), cv2.FONT_HERSHEY_PLAIN,3,(170,34,256),3)
    out.write(frame) 
    cv2.imshow("Foreground", frame)
    # Use the Esc button to quit the operation
    k = cv2.waitKey(1)
    if k%256 == 27:
        break
    
video.release()
out.release() 
cv2.destroyAllWindows()